기사시험환경 특성상, 시각화 기능이 제한된 상태로 분석 진행
   1. 1차분석: 환불여부, 카테고리 비율화, StandardScaler,StrafiedShuffle >> roc_auc_score 0.63 (RF가 가장 우수)

In [1]:
import pandas as pd
pd.options.display.float_format = '{:.4f}'.format
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/dataq/main/y_train.csv')
X_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/dataq/main/X_train.csv',encoding='euc-kr')
X_test  = pd.read_csv('https://raw.githubusercontent.com/Datamanim/dataq/main/X_test.csv',encoding='euc-kr')

In [2]:
len(X_train), len(X_test), len(y_train)

(3500, 2482, 3500)

In [3]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [4]:
X_train['gender'] = y_train['gender']
X_train.head()

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,0,68282840,11264000,6860000.0000,기타,강남점,19,3.8947,0.5270,17,0
1,1,2136000,2136000,300000.0000,스포츠,잠실점,2,1.5000,0.0000,1,0
2,2,3197000,1639000,0.0000,남성 캐주얼,관악점,2,2.0000,0.0000,1,1
3,3,16077620,4935000,0.0000,기타,광주점,18,2.4444,0.3182,16,1
4,4,29050000,24000000,0.0000,보석,본 점,2,1.5000,0.0000,85,0


In [5]:
# 여성이 train셋의 약 2/3 차지
X_train['gender'].value_counts()

0    2184
1    1316
Name: gender, dtype: int64

In [6]:
X_train['환불여부'] = X_train['환불금액'].apply(lambda x: 0 if x == 0 else 1)
X_test['환불여부'] = X_test['환불금액'].apply(lambda x: 0 if x == 0 else 1)

In [7]:
tmp = X_train[['cust_id','주구매지점']].groupby('주구매지점').count().reset_index()
tmp['cust_id'] = tmp['cust_id'] / tmp['cust_id'].sum()
tmp.columns = ['주구매지점','주구매지점Rate']
X_train = X_train.merge(tmp, on='주구매지점')
X_test = X_test.merge(tmp, on='주구매지점')

In [8]:
tmp = X_train[['cust_id','주구매상품']].groupby('주구매상품').count().reset_index()
tmp['cust_id'] = tmp['cust_id'] / tmp['cust_id'].sum()
tmp.columns = ['주구매상품','주구매상품Rate']
X_train = X_train.merge(tmp, on='주구매상품')
X_test = X_test.merge(tmp, on='주구매상품')

In [9]:
X_train.drop(['주구매상품','주구매지점'], axis=1, inplace=True)
X_test.drop(['주구매상품','주구매지점'], axis=1, inplace=True)

In [10]:
X_train = X_train.sort_values('cust_id')
X_test = X_test.sort_values('cust_id')
X_train.set_index('cust_id', inplace=True)
X_test.set_index('cust_id', inplace=True)

In [11]:
X_train.head()

,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기,gender,환불여부,주구매지점Rate,주구매상품Rate
cust_id,,,,,,,,,,,
0,68282840,11264000,6860000.0000,19,3.8947,0.5270,17,0,1,0.0414,0.1700
1,2136000,2136000,300000.0000,2,1.5000,0.0000,1,0,1,0.1354,0.0197
2,3197000,1639000,0.0000,2,2.0000,0.0000,1,1,0,0.0131,0.0157
3,16077620,4935000,0.0000,18,2.4444,0.3182,16,1,0,0.0326,0.1700
4,29050000,24000000,0.0000,2,1.5000,0.0000,85,0,0,0.3077,0.0009


In [12]:
X_train[['구매주기','gender']].groupby('구매주기').mean().values

array([[0.51957831],
       [0.37931034],
       [0.38235294],
       [0.23943662],
       [0.22429907],
       [0.25688073],
       [0.3030303 ],
       [0.26470588],
       [0.19047619],
       [0.25      ],
       [0.30681818],
       [0.2371134 ],
       [0.30588235],
       [0.24175824],
       [0.30508475],
       [0.30666667],
       [0.36486486],
       [0.32467532],
       [0.39344262],
       [0.33333333],
       [0.42857143],
       [0.33870968],
       [0.43478261],
       [0.33333333],
       [0.3125    ],
       [0.31914894],
       [0.46808511],
       [0.3902439 ],
       [0.3902439 ],
       [0.38888889],
       [0.15151515],
       [0.47826087],
       [0.42307692],
       [0.41935484],
       [0.35135135],
       [0.15      ],
       [0.40625   ],
       [0.375     ],
       [0.4375    ],
       [0.5       ],
       [0.47619048],
       [0.52380952],
       [0.47619048],
       [0.40909091],
       [0.38888889],
       [0.13333333],
       [0.4375    ],
       [0.312

In [13]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# X_train[['총구매액','최대구매액','환불금액','내점일수','내점당구매건수','구매주기']] = pd.DataFrame(scaler.fit_transform(X_train[['총구매액','최대구매액','환불금액','내점일수','내점당구매건수','구매주기']]), columns = ['총구매액','최대구매액','환불금액','내점일수','내점당구매건수','구매주기'])


In [14]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=156)
for tr_idx, te_idx in split.split(X_train, X_train['환불여부']):
    strat_train = X_train.loc[tr_idx]
    strat_test = X_train.loc[te_idx]

In [15]:
tr_x, tr_y = strat_train.drop('gender', axis=1), strat_train['gender']
val_x, val_y = strat_test.drop('gender', axis=1), strat_test['gender']

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as XGB

dt = DecisionTreeClassifier(random_state=156)
rf = RandomForestClassifier(random_state=156)
xgb = XGB.XGBClassifier()

dt.fit(tr_x, tr_y)
rf.fit(tr_x, tr_y)
xgb.fit(tr_x, tr_y)

print(dt.score(tr_x,tr_y), rf.score(tr_x,tr_y), xgb.score(tr_x,tr_y))

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:51:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
1.0 1.0 0.9796428571428571


In [17]:
from sklearn.metrics import roc_auc_score

dt_proba = dt.predict_proba(val_x)[:,1]
rf_proba = rf.predict_proba(val_x)[:,1]
xgb_proba = xgb.predict_proba(val_x)[:,1]

print("dt:", roc_auc_score(val_y, dt_proba))
print("rf:", roc_auc_score(val_y, rf_proba))
print("xgb:", roc_auc_score(val_y, xgb_proba))

dt: 0.540210980537797
rf: 0.6388174125261956
xgb: 0.612944442970705


In [18]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
총구매액,3500.0000,91919251.7971,163506530.3139,-52421520.0000,4747050.0000,28222700.0000,106507930.0000,2323180070.0000
최대구매액,3500.0000,19664241.7800,31992347.9432,-2992000.0000,2875000.0000,9837000.0000,22962500.0000,706629000.0000
환불금액,3500.0000,8289785.7914,30102039.8233,0.0000,0.0000,0.0000,2642250.0000,563753000.0000
내점일수,3500.0000,19.2537,27.1749,1.0000,2.0000,8.0000,25.0000,285.0000
내점당구매건수,3500.0000,2.8350,1.9124,1.0000,1.6667,2.3333,3.3750,22.0833
주말방문비율,3500.0000,0.3072,0.2898,0.0000,0.0273,0.2564,0.4490,1.0000
구매주기,3500.0000,20.9583,24.7487,0.0000,4.0000,13.0000,28.0000,166.0000
gender,3500.0000,0.3760,0.4844,0.0000,0.0000,0.0000,1.0000,1.0000
환불여부,3500.0000,0.3443,0.4752,0.0000,0.0000,0.0000,1.0000,1.0000
주구매지점Rate,3500.0000,0.1470,0.1138,0.0003,0.0566,0.1246,0.3077,0.3077


In [19]:
X_train.head()

,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기,gender,환불여부,주구매지점Rate,주구매상품Rate
cust_id,,,,,,,,,,,
0,68282840,11264000,6860000.0000,19,3.8947,0.5270,17,0,1,0.0414,0.1700
1,2136000,2136000,300000.0000,2,1.5000,0.0000,1,0,1,0.1354,0.0197
2,3197000,1639000,0.0000,2,2.0000,0.0000,1,1,0,0.0131,0.0157
3,16077620,4935000,0.0000,18,2.4444,0.3182,16,1,0,0.0326,0.1700
4,29050000,24000000,0.0000,2,1.5000,0.0000,85,0,0,0.3077,0.0009


In [20]:
med = X_train[['gender', '내점일수']].groupby('gender').median().iloc[0][0]
X_train['내점일수Binary'] = X_train['내점일수'].apply(lambda x: 0 if x >= med else 1)
X_test['내점일수Binary'] = X_test['내점일수'].apply(lambda x: 0 if x >= med else 1)

In [21]:
# X_train['최대구매액_총구매액'] = X_train['최대구매액'] / X_train['총구매액']
# X_test['최대구매액_총구매액'] = X_test['최대구매액'] / X_test['총구매액']

In [22]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# tmp = X_train[['총구매액','최대구매액','환불금액','내점일수','내점당구매건수', '구매주기']]
# X_train[tmp.columns] = pd.DataFrame(scaler.fit_transform(tmp), columns=tmp.columns)

In [23]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=156)
for tr_idx, te_idx in split.split(X_train, X_train[['gender','환불여부','내점일수Binary']]):
    s_train = X_train.loc[tr_idx]
    s_test = X_train.loc[te_idx]

In [24]:
tr_x, tr_y = s_train.drop('gender', axis=1), s_train['gender']
val_x, val_y = s_test.drop('gender', axis=1), s_test['gender']

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as XGB

rf = RandomForestClassifier(random_state=156)
dt = DecisionTreeClassifier(random_state=156)
xgb = XGB.XGBClassifier(random_state=156)

In [26]:
rf.fit(tr_x, tr_y)
dt.fit(tr_x, tr_y)
xgb.fit(tr_x, tr_y)

print(rf.score(tr_x,tr_y), dt.score(tr_x,tr_y), xgb.score(tr_x,tr_y))

[10:51:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9992857142857143 0.9996428571428572 0.9814285714285714


C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [27]:
rf_proba = rf.predict_proba(val_x)[:,1]
dt_proba = dt.predict_proba(val_x)[:,1]
xgb_proba = xgb.predict_proba(val_x)[:,1]

In [28]:
from sklearn.metrics import roc_auc_score

rf_score = roc_auc_score(val_y, rf_proba)
dt_score = roc_auc_score(val_y, dt_proba)
xgb_score = roc_auc_score(val_y, xgb_proba)
print(rf_score, dt_score, xgb_score)

0.6323272224204087 0.5557551922457822 0.6163002149115556


In [29]:
# 평일 방문, 주말 방문에 대한 성비
# groupby로 본 결과 남녀 주말 방문비율은 각 0.33, 0.29
# 차이가 별로 없지만, 남자는 주말 방문을 선호하고 여자는 평일이라고 가정해보자. >> 평일방문일수, 주말방문일수 만들기
X_train['평일방문가중치'] = (X_train['내점일수']*(1-X_train['주말방문비율']))-(X_train['내점일수']*X_train['주말방문비율'])
X_train

,총구매액,최대구매액,환불금액,내점일수,내점당구매건수,주말방문비율,구매주기,gender,환불여부,주구매지점Rate,주구매상품Rate,내점일수Binary,평일방문가중치
cust_id,,,,,,,,,,,,,
0,68282840,11264000,6860000.0000,19,3.8947,0.5270,17,0,1,0.0414,0.1700,0,-1.0270
1,2136000,2136000,300000.0000,2,1.5000,0.0000,1,0,1,0.1354,0.0197,1,2.0000
2,3197000,1639000,0.0000,2,2.0000,0.0000,1,1,0,0.0131,0.0157,1,2.0000
3,16077620,4935000,0.0000,18,2.4444,0.3182,16,1,0,0.0326,0.1700,0,6.5455
4,29050000,24000000,0.0000,2,1.5000,0.0000,85,0,0,0.3077,0.0009,1,2.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,3175200,3042900,0.0000,1,2.0000,1.0000,0,1,0,0.3077,0.0234,1,-1.0000
3496,29628600,7200000,6049600.0000,8,1.6250,0.4615,40,1,1,0.0700,0.0609,1,0.6154
3497,75000,75000,0.0000,1,1.0000,0.0000,0,0,0,0.0011,0.0091,1,1.0000


In [30]:
abs(X_train.corr()['gender']).sort_values(ascending=False)

gender       1.0000
내점일수Binary   0.2044
환불여부         0.2025
평일방문가중치      0.1625
내점일수         0.1550
총구매액         0.1501
환불금액         0.1143
최대구매액        0.1143
주말방문비율       0.0736
주구매상품Rate    0.0528
내점당구매건수      0.0439
구매주기         0.0415
주구매지점Rate    0.0284
Name: gender, dtype: float64

In [31]:
X_train.drop(['주구매지점Rate','구매주기','내점당구매건수','주구매상품Rate'], axis=1, inplace=True)

In [33]:
import numpy as np
tmp = np.log1p(X_train[['총구매액','최대구매액','환불금액','내점일수']])
X_train[tmp.columns] = tmp

In [34]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as XGB

rf = RandomForestClassifier(random_state=156)
dt = DecisionTreeClassifier(random_state=156)
xgb = XGB.XGBClassifier(random_state=156)

rf.fit(tr_x, tr_y)
dt.fit(tr_x, tr_y)
xgb.fit(tr_x, tr_y)

print(rf.score(tr_x,tr_y), dt.score(tr_x,tr_y), xgb.score(tr_x,tr_y))

rf_proba = rf.predict_proba(val_x)[:,1]
dt_proba = dt.predict_proba(val_x)[:,1]
xgb_proba = xgb.predict_proba(val_x)[:,1]

from sklearn.metrics import roc_auc_score

rf_score = roc_auc_score(val_y, rf_proba)
dt_score = roc_auc_score(val_y, dt_proba)
xgb_score = roc_auc_score(val_y, xgb_proba)
print(rf_score, dt_score, xgb_score)

C:\Users\user\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:51:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9992857142857143 0.9996428571428572 0.9814285714285714
0.6323272224204087 0.5557551922457822 0.6163002149115556
